In [ ]:
from reactions_manager import IsSynthesizable

In [ ]:
from rdkit.Chem import MolFromSmiles

### As described in the documentaion, there are two possible scenarios to use this module:
### a) fragmentation of an existing single acyclic bond
### b) formation of a new single acyclic bond

### NOTE: In either of theses scenarios the class 'IsSynthesizable' should be initialized twice in an identical manner with the only exception of the swap argument which should be once set as True and once as False in order to check both sides of the query bond for atom environments matching a reaction (because the algorithm is agnostic of the order in which the connecting atoms are provided as input)

### Bond fragmentation scenario

In [ ]:
mol_1 = MolFromSmiles("CNC(=O)NC") # amide / thioamide
mol_2 = MolFromSmiles("CC#CC=C") # sonogashira
mol_3 = MolFromSmiles("c1ccccc1C=Cc1ncccc1") # stille
mol_4 = MolFromSmiles("CC(=O)CC") # grignard carbonyl
mol_5 = MolFromSmiles("CC(C)(O)CC") # grignard alcohol
mol_6 = MolFromSmiles("C(C)N(C(=O)C)C(=O)C") # imide
mol_7 = MolFromSmiles("CC(C)N(C)S(=O)(=O)C") # sulfonamide (both S-N and C-N)
mol_8 = MolFromSmiles("CC(C)SC") # ether / thioether
mol_9 = MolFromSmiles("CCN(C)C") # amine
mol_10 = MolFromSmiles("c12ccc(C)cc1cc(C)n2C1=CCNCC1") # aliphatic C - aromatic N
mol_11 = MolFromSmiles("CC(=O)OC") # ester / thioester
mol_12 = MolFromSmiles("CCCC") # negishi
mol_13 = MolFromSmiles("N1(c2ccccc2)C=NC=C1") # aroN-aroC
mol_14 = MolFromSmiles("OC(=O)C(C)=C(C)c2ccccc2") # heck
mol_15 = MolFromSmiles("C1C(=O)N(C1)CC") # lactam
mol_16 = MolFromSmiles("CCSSCC") # disulfide
mol_17 = MolFromSmiles("c1ccccc1c2ccncc2") # suzuki
mol = mol_17

In [ ]:
# DETECT ANY OF THE 21 PRE-DEFINED REACTIONS
for bond in mol.GetBonds():
    left_atom = bond.GetBeginAtom()
    right_atom = bond.GetEndAtom()
    synth1 = IsSynthesizable(mol, mol, left_atom, right_atom, swap=False, track_reactions=True, debug=False)
    synth2 = IsSynthesizable(mol, mol, left_atom, right_atom, swap=True, track_reactions=True, debug=False)
    print ("Bond idx", bond.GetIdx(), "Unswapped:", synth1.detect())
    print ("Bond idx", bond.GetIdx(), "Swapped:", synth2.detect())

In [ ]:
# DETECT ONLY A SPECIFIC REACTION FOR A CERTAIN BOND (e.g. grignard carbonyl and grignard alcohol)
ketone = MolFromSmiles("c1ccccc1C(=O)CC")
for b in ketone.GetBonds():
    begin = b.GetBeginAtom()
    end = b.GetEndAtom()
    k = IsSynthesizable(ketone, ketone, begin, end, swap=False)
    print("Bond idx", b.GetIdx(), "-->", "Grignard carbonyl:", k.grignard_carbonyl(), "###", "Grignard alcohol:", k.grignard_alcohol())

### Bond formation scenario

In [ ]:
def get_next_to_r(molecule):
    for a in molecule.GetAtoms():
        if a.GetAtomicNum() == 0:
            for i, n in enumerate(a.GetNeighbors()):
                if i > 0:
                    raise ValueError
                return n

In [ ]:
mol_frag1 = MolFromSmiles("[1*]CC1=NCCN1")
mol_frag2 = MolFromSmiles("[1*]N(C)CC(=O)O")
#mol_frag3 = MolFromSmiles("[1*]c1nnc(o1)S")

atom_to_connect_1 = get_next_to_r(mol_frag1)
atom_to_connect_2 = get_next_to_r(mol_frag2)

In [ ]:
# Bond formation between the two molecules permitted 
unswapped = IsSynthesizable(mol_frag1, mol_frag2, atom_to_connect_1, atom_to_connect_2, track_reactions=True, swap=False, debug=True).detect()
# Bond formation between the two molecules not permitted
swapped = IsSynthesizable(mol_frag1, mol_frag2, atom_to_connect_1, atom_to_connect_2, track_reactions=True, swap=True, debug=True).detect()

In [ ]:
unswapped, swapped